# Sentiment Analysis on Reddit Data

In [ ]:
import re
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,col
from pyspark.sql.types import *
#Connect to the cluster
# New API
spark = SparkSession\
        .builder\
        .master("spark://192.168.2.77:7077") \
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .appName("pa_test1")\
        .getOrCreate()

# Old API (RDD)
sc = spark.sparkContext

sc.setLogLevel("INFO")

## Load sentiment files

In [2]:
nlines = sc.textFile("hdfs://192.168.2.77:9000/user/ubuntu/negative-words.txt")
plines = sc.textFile("hdfs://192.168.2.77:9000/user/ubuntu/positive-words.txt")

## Load comment data from hdfs and select columns of interest

In [3]:
df = spark.read.json("hdfs://192.168.2.77:9000/user/ubuntu/sample_data.json")
data_clean = df.select("subreddit", "body", "score", "controversiality", "created_utc")

## Prepare negative and positive word data and broadcast it to the cluster

In [4]:
def compile_regexp(word_list):
    re_string = "[\s\W]("
    for word in word_list:
        re_string += (re.escape(word) + "|")
    re_string = re_string[0:-1] + ")[\s\W]"
    return re.compile(re_string, re.IGNORECASE)

negative = compile_regexp(nlines.collect())
sc.broadcast(negative)
positive = compile_regexp(plines.collect())
sc.broadcast(positive)

## Define udf(s) to return the ratio of negativity and positivity for each comment

In [5]:
def match_negative(comment, wc):
    return len(negative.findall(comment))/wc

def match_positive(comment, wc):
    return len(positive.findall(comment))/wc

def count_words(comment):
    return len(comment.split())

udf_match_negative = udf(match_negative, DoubleType())
udf_match_positive = udf(match_positive, DoubleType())
udf_count_words = udf(count_words, IntegerType())

In [17]:
data_clean.printSchema()

root
 |-- subreddit: string (nullable = true)
 |-- body: string (nullable = true)
 |-- score: long (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: long (nullable = true)



## Create columns negativity, positivity and wordcount

In [7]:
w_count = data_clean.withColumn('wordcount', udf_count_words('body'))
negativity = w_count.withColumn('negativity', udf_match_negative('body', 'wordcount'))
n_p_df = negativity.withColumn('positivity', udf_match_positive('body', 'wordcount'))
filtered = n_p_df.select("subreddit", "score", "controversiality", "wordcount", "negativity", "positivity", "created_utc").filter("positivity != 0 and negativity != 0")
filtered.show()
#n_p_df.head()

+---------------+-----+----------------+---------+--------------------+--------------------+-----------+
|      subreddit|score|controversiality|wordcount|          negativity|          positivity|created_utc|
+---------------+-----+----------------+---------+--------------------+--------------------+-----------+
|      EchoArena|    1|               0|       55| 0.03636363636363636| 0.03636363636363636| 1506816000|
|     The_Donald|    2|               0|        9|  0.2222222222222222|  0.1111111111111111| 1506816000|
|       totalwar|    3|               0|      192|0.057291666666666664|             0.03125| 1506816000|
|   tvcrossovers|    1|               0|       49|0.061224489795918366|0.061224489795918366| 1506816000|
|     realmadrid|   10|               0|       13| 0.15384615384615385| 0.07692307692307693| 1506816000|
|            CFB|    1|               0|       15| 0.06666666666666667| 0.13333333333333333| 1506816000|
|     edc_raffle|    1|               0|      221| 0.01

## Group by subreddit's average positivity and negativity

In [9]:
#Group By subreddit
df1 = filtered.groupBy("created_utc", "subreddit").avg("positivity", "negativity")

#find "happiest" subreddit of the day 
highest_average_positive = df1.agg({"avg(positivity)": "max"}).first()[0]


## Define Happiness Ratio as the avg(pos/neg)

In [10]:
#Create Happiness Ration Col 
df2 = df1.withColumn("Happiness Ratio", df1["avg(positivity)"] / df1["avg(negativity)"])

In [11]:
df2.show()

+-----------+---------------+--------------------+--------------------+-------------------+
|created_utc|      subreddit|     avg(positivity)|     avg(negativity)|    Happiness Ratio|
+-----------+---------------+--------------------+--------------------+-------------------+
| 1506816209|            CFB| 0.09803921568627451|  0.0196078431372549|                5.0|
| 1506816280|  todayilearned|0.041666666666666664| 0.08333333333333333|                0.5|
| 1506816291|            DFO| 0.09090909090909091|0.045454545454545456|                2.0|
| 1506816001|    techtheatre| 0.09090909090909091|0.030303030303030304|                3.0|
| 1506816011|  linuxhardware| 0.03571428571428571| 0.08928571428571429|0.39999999999999997|
| 1506816018|            WTF| 0.23076923076923078| 0.07692307692307693|                3.0|
| 1506816041|           news| 0.05263157894736842| 0.05263157894736842|                1.0|
| 1506816047|      starbucks|0.030303030303030304| 0.06060606060606061|         

## Convert unix time to "MM-dd-yyyy HH:mm:ss" Date format

In [18]:
#Convert unix timestamp to "MM-dd-yyyy HH:mm:ss" Date format
from pyspark.sql.functions import *
df3 = df2.withColumn("created_utc", (from_unixtime(col("created_utc"),"MM-dd-yyyy HH:mm:ss").alias("created_utc")))

#find max hour 
#df3.agg({"created_utc": "max"}).first()[0]

In [13]:
df3.take(10)

[Row(created_utc='10-01-2017 00:00:01', subreddit='techtheatre', avg(positivity)=0.09090909090909091, avg(negativity)=0.030303030303030304, Happiness Ratio=3.0),
 Row(created_utc='10-01-2017 00:00:11', subreddit='linuxhardware', avg(positivity)=0.03571428571428571, avg(negativity)=0.08928571428571429, Happiness Ratio=0.39999999999999997),
 Row(created_utc='10-01-2017 00:00:18', subreddit='WTF', avg(positivity)=0.23076923076923078, avg(negativity)=0.07692307692307693, Happiness Ratio=3.0),
 Row(created_utc='10-01-2017 00:00:41', subreddit='news', avg(positivity)=0.05263157894736842, avg(negativity)=0.05263157894736842, Happiness Ratio=1.0),
 Row(created_utc='10-01-2017 00:00:47', subreddit='starbucks', avg(positivity)=0.030303030303030304, avg(negativity)=0.06060606060606061, Happiness Ratio=0.5),
 Row(created_utc='10-01-2017 00:00:49', subreddit='todayilearned', avg(positivity)=0.04040404040404041, avg(negativity)=0.06060606060606061, Happiness Ratio=0.6666666666666667),
 Row(created_u

In [14]:
from pyspark.sql import functions as F
from pyspark.sql.functions import *
df4 = df3.withColumn("Hour", hour(F.to_timestamp("created_utc","MM-dd-yyyy HH:mm:ss")))

In [15]:
#df1 = filtered.groupBy("created_utc", "subreddit").avg("positivity", "negativity")
df5 = df4.groupBy("Hour").avg("Happiness Ratio")

## Happiness ration of reddit each hour

In [16]:
df5.show()

+----+--------------------+
|Hour|avg(Happiness Ratio)|
+----+--------------------+
|   0|  1.4285350419065057|
+----+--------------------+

